## Homework

1. Complete the Python implementation of the backpropagation exercise in the **Backpropagation** section here above (cell `# try it in Python as homework!`)
    - Create the calculations for obtaining $y$ in PyTorch **using only PyTorch methods and routines**
    - Calculate the gradient
    - Check the values of the gradients and see if it is correct w.r.t. the manual calculations
2. Given the multilayer perceptron defined during the exercises from lab 1:
    - Create 10 random datapoints (with any function you wish, it can be `rand`, `randn`...) and feed them into the network
    - Given the output, calculate the Cross-Entropy loss with respect to the ground truth $[1,2,3,4,1,2,3,4,1,2]$ (classes from 1 to 4). Cross-Entropy loss:
        
        $$ CE(\mathbf{y}, \hat{\mathbf{y}}) = - \frac{1}{N}\sum_{i=1}^{N} {y}_i \odot \log(\hat{y_i})$$
        
        where $y_i$ is the one-hot encoding of the $i$-th datapoint. For instance, $y_1 = [1,0,0,0]$.
        **_Note: there is an extremely handy PyTorch function for getting a one-hot encoding out of a vector, so don't try anything fancy._**
    - Backpropagate the error along the network and inspect the gradient of the parameters connecting the input layer and the first hidden layer.
3. Execute the python script `utils/randomized_backpropagation_formula.py`. This creates a formula $f(\mathbf{x})$ with randomized operators and values. Create the computational graph from this formula, do (by hand) the forward pass, then calculate (by hand) $\nabla f(\mathbf{x})$ using the backward gradient computation. Do the same calculation on PyTorch to check the correctness of your calculations. _Note: The formula created by this script is linked to your name and surname, which you have to input before_. The solution to this exercise _should_ be submitted as a scan/good quality picture of a piece of paper (or you can do it on a touch screen and submit the image...), but other formats are acceptable as well.


## 1) **Backpropagation**

Let us suppose we have the following calculation

$\mathbf{x} = [1,~2,~-1,~3,~5]$

$ y = f(\mathbf{x}) = \log\{[\exp (x_1 * x_2 )]^2 + \sin (x_3 + x_4 + x_5) \cdot x_5\}$

Find

$\nabla f(\mathbf{x})$

In [2]:
import torch 

# Manual calculation of the output gave us y = 4.06 and x.grad = (3.76, 1.88, 0.65, 0.65, 0.076)
x = torch.tensor([1,2,-1,3,5], dtype=torch.float32, requires_grad=True)
y = ((x[0]*x[1]).exp()**2 + (x[2] + x[3] + x[4]).sin()*x[4]).log()
y.backward()

print(f'Output for {y=}')
print(f'Gradient for {x.grad=}')

Output for y=tensor(4.0584, grad_fn=<LogBackward0>)
Gradient for x.grad=tensor([3.7730, 1.8865, 0.0651, 0.0651, 0.0765])


## 2) **Cross-Entropy Loss**

In [3]:
class MLP_hw(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(5, 11, bias=False),
            torch.nn.ReLU(),
            torch.nn.Linear(11, 16, bias=False),
            torch.nn.ReLU(),
            torch.nn.Linear(16, 13, bias=False),
            torch.nn.ReLU(),
            torch.nn.Linear(13, 8, bias=False),
            torch.nn.ReLU(),
            torch.nn.Linear(8, 4, bias=False),
            torch.nn.Softmax(dim=1)
        )

    def forward(self, X):
        return self.layers(X)

my_model = MLP_hw()

In [4]:
input = torch.randn((10, 5))
out_model_prob = my_model(input)
print(f'{out_model_prob=}')                                           # These are the prob. for the outputs coming from the model

out_model_prob=tensor([[0.2506, 0.2498, 0.2506, 0.2490],
        [0.2514, 0.2499, 0.2503, 0.2484],
        [0.2521, 0.2497, 0.2498, 0.2484],
        [0.2509, 0.2499, 0.2501, 0.2491],
        [0.2554, 0.2483, 0.2495, 0.2468],
        [0.2540, 0.2487, 0.2503, 0.2470],
        [0.2514, 0.2500, 0.2508, 0.2478],
        [0.2521, 0.2502, 0.2511, 0.2466],
        [0.2515, 0.2503, 0.2508, 0.2474],
        [0.2529, 0.2496, 0.2495, 0.2480]], grad_fn=<SoftmaxBackward0>)


In [7]:
ground_truth = torch.tensor([1, 2, 3, 4, 1, 2, 3, 4, 1, 2], dtype=torch.long) - 1
ground_truth_ohe = torch.nn.functional.one_hot(ground_truth, num_classes=4)
print(ground_truth_ohe)

tensor([[1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
        [1, 0, 0, 0],
        [0, 1, 0, 0]])


In [ ]:
def CrossEntropyLoss(y, y_hat):
    return -torch.sum(y * torch.log(y_hat))/y.shape[0]
   

CEL = CrossEntropyLoss(ground_truth_ohe, out_model_prob)
print(f'Cross-Entropy Loss: {CEL}')
CEL.backward()
print(f'Gradient for {my_model.layers[0].weight.grad=}')

Cross-Entropy Loss: 1.3862920999526978
Gradient for my_model.layers[0].weight.grad=tensor([[-7.8853e-04,  2.7982e-03, -1.8420e-03,  1.9714e-03,  4.6874e-03],
        [ 6.1023e-04, -4.3326e-03,  7.7628e-05, -4.3527e-03, -6.0485e-03],
        [-1.8783e-03,  8.4448e-04, -6.5822e-04, -3.7221e-04,  2.2365e-03],
        [-4.6236e-03, -2.7560e-03, -2.2282e-03, -7.9049e-04, -7.2914e-04],
        [-1.5556e-03,  3.9361e-03,  7.3019e-03,  1.4282e-03,  2.8106e-03],
        [ 2.6757e-03,  1.7487e-04, -1.4265e-03,  1.5886e-03,  2.2141e-03],
        [-5.1425e-03,  2.9732e-03,  2.2091e-03, -1.8388e-03, -1.7869e-03],
        [ 8.1484e-03,  3.9976e-03, -2.8636e-03,  5.9499e-04, -1.7781e-03],
        [-1.3405e-03, -2.4607e-05, -8.0396e-04, -1.2235e-03,  1.7071e-03],
        [-5.6152e-03,  2.3406e-03,  2.3747e-05, -2.6906e-03,  9.5219e-04],
        [-5.1180e-03,  7.0050e-03,  3.5651e-03,  6.5166e-03,  8.7383e-03]])


## 3) **Random Function**
My function was $$\cos(((x1 + x2)^3 + \frac{1}{(x3 / x4)}) - x5)$$
With the initial input: $$\vec{x} = (-3, -3, 1, -1, 4)$$

In [ ]:
X = torch.tensor([-3, -3, 1, -1, 4], requires_grad=True, dtype=torch.float32)
z = ((X[0:2].sum()**3 + (X[3]/X[2])) - X[4]).cos()
z.backward()
print(f'{z=}')
print(f'{X.grad=}')

z=tensor(0.4638, grad_fn=<CosBackward0>)
X.grad=tensor([95.6814, 95.6814,  0.8859,  0.8859, -0.8859])


In the pdf there is the solution written by hand, we can see that they are approximately the same